In [1]:
import pandas as pd
import numpy as np


In [2]:
hr_train = pd.read_csv('C:/Users/Prathmesh/Desktop/Python/train_hr.csv',
                       header = 0)

hr_train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [3]:
hr_train.shape

(54808, 14)

In [4]:
hr_train=hr_train.drop('employee_id',axis=1)

In [5]:
hr_train.isnull().sum()

department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [6]:
hr_train.previous_year_rating.value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
Name: previous_year_rating, dtype: int64

In [7]:
for value in ['education', 'previous_year_rating']:
    hr_train[value].fillna(hr_train[value].mode()[0],inplace=True)


In [8]:
hr_train.isnull().sum()

department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
dtype: int64

In [9]:
colname = ['department',
          'region', 'education',
          'gender','recruitment_channel']
colname

['department', 'region', 'education', 'gender', 'recruitment_channel']

In [10]:
# For preprocessing the data
from sklearn import preprocessing

le=preprocessing.LabelEncoder()

for x in colname:
     hr_train[x]=le.fit_transform(hr_train[x])


In [11]:
hr_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,7,31,2,0,2,1,35,5.0,8,1,0,49,0
1,4,14,0,1,0,1,30,5.0,4,0,0,60,0
2,7,10,0,1,2,1,34,3.0,7,0,0,50,0
3,7,15,0,1,0,2,39,1.0,10,0,0,50,0
4,8,18,0,1,0,1,45,3.0,2,0,0,73,0


In [12]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = hr_train[hr_train.is_promoted==0]
df_minority = hr_train[hr_train.is_promoted==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=50140,    # to match majority class
                                 random_state=10) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.is_promoted.value_counts()

1    50140
0    50140
Name: is_promoted, dtype: int64

In [13]:
X = df_upsampled.values[:,:-1]
Y = df_upsampled.values[:,-1]



In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)

X = scaler.transform(X)
print(X)

[[ 0.80095407  1.76473212  1.54534627 ...  0.98195379 -0.2672541
  -1.23779348]
 [-0.38713063 -0.1773577  -0.65882154 ... -1.01837786 -0.2672541
  -0.48025744]
 [ 0.80095407 -0.63432001 -0.65882154 ... -1.01837786 -0.2672541
  -1.16892656]
 ...
 [ 1.19698231 -0.06311712  1.54534627 ...  0.98195379 -0.2672541
   0.75934699]
 [ 0.40492584  0.39384519 -0.65882154 ...  0.98195379 -0.2672541
   1.31028229]
 [-1.97124358 -0.1773577  -0.65882154 ... -1.01837786 -0.2672541
   1.3791492 ]]


In [15]:
from sklearn.model_selection import train_test_split

#Split the data into test and train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,
                                                    random_state=10)  


In [16]:
#predicting using the Decision_Tree_Classifier
from sklearn.tree import DecisionTreeClassifier

model_DecisionTree=DecisionTreeClassifier(random_state=10)

#fit the model on the data and predict the values
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score,\
classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print("Classification report: ")

print(classification_report(Y_test,Y_pred))

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)


[[14000  1015]
 [    0 15069]]
Classification report: 
              precision    recall  f1-score   support

         0.0       1.00      0.93      0.97     15015
         1.0       0.94      1.00      0.97     15069

    accuracy                           0.97     30084
   macro avg       0.97      0.97      0.97     30084
weighted avg       0.97      0.97      0.97     30084

Accuracy of the model:  0.9662611354873022


In [18]:
hr_test = pd.read_csv(r'C:/Users/Prathmesh/Desktop/Python/test_hr.csv',
                       header = 0)

hr_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [19]:
hr_test=hr_test.drop('employee_id',axis=1)

In [20]:
hr_test.isnull().sum()

department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [21]:
for value in ['education', 'previous_year_rating']:
    hr_test[value].fillna(hr_test[value].mode()[0],inplace=True)


In [22]:
hr_test.isnull().sum()

department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64

In [23]:
colname = ['department',
          'region', 'education',
          'gender','recruitment_channel']
colname

['department', 'region', 'education', 'gender', 'recruitment_channel']

In [24]:
# For preprocessing the data
from sklearn import preprocessing

le=preprocessing.LabelEncoder()

for x in colname:
     hr_test[x]=le.fit_transform(hr_test[x])


In [25]:
hr_test.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8,18,0,1,2,1,24,3.0,1,1,0,77
1,2,28,0,0,0,1,31,3.0,5,0,0,51
2,7,4,0,1,0,1,31,1.0,4,0,0,47
3,5,11,0,0,0,3,31,2.0,9,0,0,65
4,1,21,0,1,2,1,30,4.0,7,0,0,61


In [26]:
X_test_new=hr_test

In [27]:

X_test_new = scaler.transform(X_test_new)


In [28]:
Y_pred=model_DecisionTree.predict(X_test_new)

In [29]:
Y_pred=Y_pred.astype(int)

In [31]:
test_data=pd.read_csv(r'C:\Users\Prathmesh\Desktop\Python\test_hr.csv')
test_data["Y_predictions"]=Y_pred
test_data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,Y_predictions
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77,0
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51,0
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47,0
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65,0
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61,0


In [33]:
test_data.Y_predictions.value_counts()

0    21134
1     2356
Name: Y_predictions, dtype: int64

In [ ]:
"""
test_size=0.2
0    21235
1     2255
"""

"""
test_size=0.3
0    21134
1     2356
"""

In [35]:
test_data.to_excel('C:/Users/Prathmesh/Desktop/Python/output.xlsx')

In [37]:
from sklearn import tree
with open("C:/Users/Prathmesh/Desktop/Python/model_DecisionTree.txt", "w") as f:

    f = tree.export_graphviz(model_DecisionTree, feature_names=hr_train.columns[:-1],
    out_file=f)

#generate the file and upload the code in webgraphviz.com to plot the decision tree